# Table of Contents
 <p><div class="lev1"><a href="#VARIABLE-DESCRIPTIONS:"><span class="toc-item-num">1&nbsp;&nbsp;</span>VARIABLE DESCRIPTIONS:</a></div>

# VARIABLE DESCRIPTIONS:

* survival        Survival            (0 = No; 1 = Yes)
* pclass          Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name            Name
* sex             Sex
* age             Age
* sibsp           Number of Siblings/Spouses Aboard
* parch           Number of Parents/Children Aboard
* ticket          Ticket Number
* fare            Passenger Fare
* cabin           Cabin
* embarked        Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.

In [65]:
# This Python 3 environment comes with many helpful analytics libraries 
# installed
# It is defined by the kaggle/python docker image: 
# https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from IPython.display import display
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import f1_score, accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer, OneHotEncoder, StandardScaler,
    MinMaxScaler, MaxAbsScaler, Imputer
)
from sklearn.svm import SVC

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) 
# will list the files in the input directory

import sh
print(sh.ls('./input'))

genderclassmodel.csv  gendermodel.csv  test.csv  train.csv  train.csv.ori



In [75]:
Y_test =  pd.read_csv('./input/genderclassmodel.csv')
test =  pd.read_csv('./input/test.csv')
train =  pd.read_csv('./input/train.csv')

print('test set has %s rows and %s columns.' % test.shape)
print('test set has %s rows and %s columns.' % Y_test.shape)
print('train set has %s rows and %s columns.' % train.shape)

test set has 418 rows and 11 columns.
test set has 418 rows and 2 columns.
train set has 891 rows and 12 columns.


In [76]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [99]:
def prepare_df(df, p_unique=0.2):
    df = df.copy()
    
    size = len(df)
    
    df = pd.get_dummies(df, drop_first=True)
    cols = list(df.columns)
    
    for c in cols:
        if df[c].dtype == object:
            if df[c].nunique()/size > p_unique:
                df.drop(c, axis=1, inplace=True)
                continue
            
    return df

In [100]:
pca = make_pipeline(
    Imputer(missing_values='NaN', strategy='mean', axis=0),
    StandardScaler(),
    PCA(random_state=42)
)

pca.fit_transform(prepare_df(train.drop('Survived', axis=1)), train.Survived);

In [96]:
pca.get_params()

{'imputer': Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0),
 'imputer__axis': 0,
 'imputer__copy': True,
 'imputer__missing_values': 'NaN',
 'imputer__strategy': 'mean',
 'imputer__verbose': 0,
 'memory': None,
 'pca': PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
   svd_solver='auto', tol=0.0, whiten=False),
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': 42,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'steps': [('imputer',
   Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)),
  ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('pca',
   PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
     

In [53]:
train_ = train.copy()

train_.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
# train_.dropna(inplace=True)

X_train = prepare_df(train_.drop('Survived', axis=1))
y_train = train_[['Survived']]

In [54]:
test_ = pd.concat([Y_test, test.iloc[:, 1:]], axis=1)

test_.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
# test_.dropna(inplace=True)

X_test = prepare_df(test_.drop('Survived', axis=1))
y_test = test_[['Survived']]

In [7]:
test_.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [55]:
print('X_test set has %s rows and %s columns.' % X_test.shape)
print('y_test set has %s rows and %s columns.' % y_test.shape)
print('X_train set has %s rows and %s columns.' % X_train.shape)
print('Y_train set has %s rows and %s columns.' % y_train.shape)

X_test set has 418 rows and 8 columns.
y_test set has 418 rows and 1 columns.
X_train set has 891 rows and 8 columns.
Y_train set has 891 rows and 1 columns.


In [59]:
# , MaxAbsScaler
pipeline_rfc = make_pipeline(
    Imputer(missing_values='NaN', strategy='mean', axis=0),
    MaxAbsScaler(),
    RandomForestClassifier(random_state=42)
)

pipeline_svc = make_pipeline(
    Imputer(missing_values='NaN', strategy='mean', axis=0),
    MaxAbsScaler(),
    SVC()
)

In [60]:
'''
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
f1 score
0.768558951965
precision score
1.0
accuracy score
0.873205741627


<class 'sklearn.svm.classes.SVC'>
f1 score
0.873015873016
precision score
0.990990990991
accuracy score
0.923444976077
'''
for title, model in {
    'RandomForestClassifier': pipeline_rfc, 
    'SVC': pipeline_svc
}.items():
    print('\n%s' % model)
    model.fit(X_train, y_train)

    predicted = model.predict(X_test)
    print('f1 score')
    print(f1_score(y_test, predicted))
    print('precision score')
    print(precision_score(y_test, predicted))
    print('accuracy score')
    print(accuracy_score(y_test, predicted))
    f_name = 'output/%s.csv' % title
    
    # np.savetxt(f_name, predicted)
    # print('%s saved!' % f_name)


Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('maxabsscaler', MaxAbsScaler(copy=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_lea...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])
f1 score
0.7491408934707905
precision score
0.7266666666666667
accuracy score
0.8253588516746412

Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('maxabsscaler', MaxAbsScaler(copy=True)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
f1 score
0.962457337883959
precision score
0.9276315789473685
acc

/home/xmn/miniconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/home/xmn/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
